# Encriptación Simétrica

Con este cuaderno se pretende exponer los conceptos claves de la encriptación simétrica, la cual debemos de saber que consiste en aquella en la que encriptamos el mensaje mediante una clave secreta la cual deberá de ser compartida por el emisor y el receptor del mensaje.
![None](https://media.discordapp.net/attachments/886925237990617119/908806236894552074/EncriptacionSimetrica.png)

A la hora de trabajar con encriptación simétrica existen dos tipos de algoritmos aquellos en los cuales vamos cifrando el mensaje, generando bloques los cuales serán transmitidos y que para poder desencriptar el mensaje necesario descencriptar cada uno de los bloques transmitidos. 

En el otro caso, tenemos los **Stream Ciphers**, en los cuales se realizará la encriptación bit a bit.

Para trabajar con este cuaderno será necesario tener instalado el paquete **[PyCryptodome](https://www.pycryptodome.org/en/latest/)** de Python, por ello en caso de no tenerlo instálelo de la siguiente manera:

Una vez realizado esto podremos importa las funciones necesarias para realizar la encriptación simétrica de bloques.

## Modos clásicos de operación para encriptación simétrica de bloque

A la hora de trabajar con los modos clásicos se nos propone utilizar el algoritmo \textbf{AES}, para comprender el funcionamiento de la encriptación. 

Para ello, lo primero que haremos será importar la clase AES, y así poder trabajar con una instancia de esta clase. A esta clase le deberemos de pasar en el constructor la ```clave'' y el método de encriptación a emplear. 

In [1]:
from base64 import b64encode
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes

ModuleNotFoundError: No module named 'Crypto'

Deberemos de indicar cual es el mensaje que deseamos encriptar, además de generar una clave de forma aleatoria:


In [2]:
mensaje = "Hola mundo"
mensajeBinario = mensaje.encode()
clave = get_random_bytes(16)
cipher = AES.new(clave, AES.MODE_CBC)
ct_bytes = cipher.encrypt(pad(mensajeBinario,AES.block_size))

NameError: name 'get_random_bytes' is not defined

De esta forma en ct_bytes tendremos el texto cifrado en binario, para poder obtener el texto en un formato  UTF-8, podemos utilizar **b64encode**, para obtener ese valor en codificación **utf-8**.

In [ ]:
iv = b64encode(cipher.iv).decode('utf-8')
ct = b64encode(ct_bytes).decode('utf-8')

Si mostramos ambas variables podemos ver tanto el vector de inicialización como el texto encriptado.


In [ ]:
    print (f"{iv}")
    print (f"{ct}")

### Desencriptación

Para desencriptar lo que deberemos de hacer será en primer lugar, decodificar el mensaje y el vector de inicialización, ya que lo tenemos en formato **Base64**. Para ello usaremos ***b64decode***.

Tras esto lo que haremos será crear nuestro objeto de la clase AES, el cual utilizaremos para desencriptar el mensaje.

In [ ]:
ivCifrado = iv
cvCifrafo = ct
iv = b64decode(iv)
ct = b64decode(ct)
cipher = AES.new(clave, AES.MODE_CBC, iv)
  

Una vez creado el objeto, utilizaremos la función **decrypt**, a la cual le pasaremos el mensaje, y mediante la función **unpad**, descomprimiremos el block de AES. De esta forma el mensaje se encontrará guardado en la variable pt.

In [ ]:
pt = unpad(cipher.decrypt(ct), AES.block_size)
print("Mensaje desencriptado: ", pt)

Una vez hecho esto ya tendríamos el texto descifrado. En caso de error nos saltaría la excepción de **KeyError**.

Si deseamos ejecutar todo este fragmento en conjunto sería:

In [ ]:
try:
    iv = b64decode(ivCifrado)
    ct = b64decode(ctCifrado)
    cipher = AES.new(key, AES.MODE_CBC, iv)
    pt = unpad(cipher.decrypt(ct), AES.block_size)
    print("The message was: ", pt)
except (ValueError, KeyError):
    print("Incorrect decryption")

### Encriptación de un fichero

Si deseamos encriptar un fichero, como sabemos deberemos de ir leyendo línea a línea del fichero e ir encriptando el contenido de dicha línea. 

En este caso se nos propone conocer el funcionamiento de los modos de encriptación \textbf{CBC y CTR}, los cuales podemos diferenciar con la siguiente explicación:

- El modo **CBC**, consiste en una encriptación por bloques, en la cual se tendrá que utilizar un IV aleatorio, cada vez que se encripte un mensaje. En este caso se necesitará rellenar los bloques a transmitir con un tamaño fijo para todos. 
- El modo **CTR**, consiste en un modo de cifrado de flujo, en este caso lo que hace es mediante una secuencia pseudoaleatoria, independiente del texto, se obtendrán los diferentes IV para encriptar. 

Una vez entendidos podemos trabajar con ellos, en primer lugar deberemos de importar los mismos módulos utilizados anteriormente. 


In [ ]:
    CBCActivo = True
    CTRActivo = False

In [ ]:
Estas variables las utilizaremos para determinar que modo deseamos utilizar:

In [ ]:
try:
    fichero = open (nombreFicheroEntrada,'r')
    ficheroEncriptado = open (nombreFicheroDestino,'w')
    if CBCActivo is True:
        cipher = AES.new(key, AES.MODE_CBC)   
    elif CTRActivo is True:
        cipher = AES.new(key, AES.MODE_CTR)
    for linea in fichero:
        linea = linea[:linea.find('\\')]
        #Encriptamos el dato
    
        if CBCActivo is True:
            lineaEncriptadaBytes = cipher.encrypt(pad(linea, AES.block_size))            
        elif CTRActivo is True:
            lineaEncriptadaBytes = cipher.encrypt(linea)
            
        lineaEncriptada = b64encode(lineaEncriptadaBytes).decode('utf-8')
        ficheroEncriptado.write(lineaEncriptada)
            
finally:
    fichero.close()
    ficheroEncriptado.close()

En primer lugar deberemos de abrir los ficheros, tanto el que deseamos encriptar como en el cual deseamos guardar. Posteriormente crearemos nuestro objeto AES, con el cual realizaremos la encriptación.

Posteriormente recorreremos el fichero, eliminando a cada línea el ''\n'', posteriormente, encriptaremos el dato, dependiendo del modo Activo. Para el caso del \textbackslash{CBC} deberemos de utilizar **pad**, para especificar el contenido con el tamaño de bloque.


Cosa que no ocurre con **CTR**.

Posteriormente codificaremos el mensaje encrptado en formato **utf-8**. Y escribiendo en el fichero destino.

Si deseamos desencriptar el fichero en primer lugar necesitaremos el IV utilizado y la clave. Para ello podemos mostrarlos por pantalla o guardarlos en un fichero json.

In [ ]:
iv = b64encode(cipher.iv).decode('utf-8')
key = key.hex()
result = {}
result["iv"] = iv
result["key"] = key
print (iv)
print (key)
    
with open('data.json', 'w') as outfile:
    json.dump(result, outfile)

In [ ]:
#### Desencriptación de un fichero

A la hora de desencriptar deberemos de conocer cual fue el modo utilizado para encriptar. Además de utilizar la clave y el IV, el cual cargaremos del fichero json.

In [ ]:
try:
    with open("data.json") as json_data:
        b64 = json.load(json_data)
        print(b64)
    iv = b64decode(b64['iv'])
    cipher = AES.new(key, AES.MODE_CBC, iv)
    try:
        fichero = open (nombreFicheroEntrada,'r')
        ficheroEncriptado = open (nombreFicheroDestino,'w')
        if CBCActivo is True:
            cipher = AES.new(key, AES.MODE_CBC)   
        elif CTRActivo is True:
            cipher = AES.new(key, AES.MODE_CTR)
        for linea in fichero:
            linea = linea[:linea.find('\\')]
            #Encriptamos el dato    
            linea = b64decode(linea)
            if CBCActivo is True:
                lineaDesencriptadaBytes = cipher.decrypt(unpad(linea, AES.block_size))            
            elif CTRActivo is True:
                lineaDesencriptadaBytes = cipher.decrypt(linea)
                
            ficheroEncriptado.write(lineaDesencriptada)
                
    finally:
        fichero.close()
        ficheroEncriptado.close()
    
    
    print("The message was: ", pt)
except (ValueError, KeyError):
    print("Incorrect decryption")

Como podemos ver se utiliza la función **decrypt** para desencriptar la información. Anteriormente debemos de decodificar el mensaje.


### Encriptación Autentificada con Datos Asociados

En esta parte opcional se pretende exponer el modo **EAX** del algoritmo AES, el cual nos permitirá esta forma de encriptación. Para ello será necesario cambiar el modo que le pasamos a la hora de crear la instancia
del objeto AES. Y posteriormente utilizar la función de encrypt_and_digest, para obtener el texto
cifrado.

Para descifrar utilizaremos la función decrypt_and_verify.

El siguiente trozo de código nos permite encriptar el mensaje, para ello crearemos una clave de forma aleatoria, y a partir de ella crearemos la instancia del objeto AES con el modo EAX. Posteriormente utilizaremos la función **encrypt_and_digest**, como ya hemos indicado y finalmente lo que haremos será volcar la información al fichero.

In [ ]:
key = get_random_bytes(16)
data="Hola mundo".encode("utf-8")
cipher = AES.new(key, AES.MODE_EAX)
ciphertext, tag = cipher.encrypt_and_digest(data)
file_out = open("encrypted.bin", "wb")
[ file_out.write(x) for x in (cipher.nonce, tag, ciphertext) ]
print(key.hex())

Para el caso de desencriptar sería de la siguiente forma:

In [ ]:
i = key.hex() #Para poder coger la clave correcta 
key= bytes.fromhex(i)
file_in = open("encrypted.bin", "rb")
nonce, tag, ciphertext = [ file_in.read(x) for x in (16, 16, -1) ]
# let's assume that the key is somehow available again
cipher = AES.new(key, AES.MODE_EAX, nonce)
data = cipher.decrypt_and_verify(ciphertext, tag)
print(data.decode())

Una vez visto todo esto ya podemos trabajar con la encriptación de un fichero completo. Para ello como siempre leeremos el fichero e iremos utilizando el objeto AES, para encriptar o desencriptar.

In [ ]:
nombreFicheroEntrada = "hola.txt"

nombreFicheroDestino = "hola.bin"
    
key = get_random_bytes(16)
    
cipher = AES.new(key, AES.MODE_EAX)

In [ ]:
data = ""
try:
    fichero = open (nombreFicheroEntrada,'r')
    ficheroEncriptado = open( nombreFicheroDestino + ".bin", "wb")
    
    for linea in fichero:
        #Encriptamos el dato
        data+=linea
    
    ciphertext, tag = cipher.encrypt_and_digest(data.encode())
    [ ficheroEncriptado.write(x) for x in (cipher.nonce, tag, ciphertext) ]
    
    print(key.hex())        
finally:
    fichero.close()
    ficheroEncriptado.close()

Para el caso de desencriptación sería de la misma forma: 

In [ ]:
nombreFicheroEntrada = "hola.bin"
    
nombreFicheroDestino = "holaDesencriptado.txt"
    
i = key.hex()
key= bytes.fromhex(i)
    
cipher = AES.new(key, AES.MODE_EAX)
     
#Debemos de leer todo el contenido del fichero y guardarlo en la variable ciphertext
data = ""
try:
    fichero = open (nombreFicheroEntrada,'rb')
    ficheroDesencriptado = open( nombreFicheroDestino, "w")
        
    nonce, tag, ciphertext = [ fichero.read(x) for x in (16, 16, -1) ]
    
    cipher = AES.new(key, AES.MODE_EAX, nonce)
    data = cipher.decrypt_and_verify(ciphertext, tag)
    print(data.decode())
    ficheroDesencriptado.write(data.decode())
            
finally:
    fichero.close()
    ficheroDesencriptado.close()

Como podemos ver para encriptar los ficheros hemos recorrido los ficheros y almacenado el contenido en una variable y posteriormente encriptado este contenido. Esto podría dar problemas a la hora de que necesitaríamos almacenar toda la información del fichero en memoria para realizar este proceso. Siendo muy costoso a nivel de memoria.